# Lab 1 - Training a Machine Learning Model

In this lab you will setup the Azure Machine Learning service from code and create a classical machine learning model that logs metrics collected during model training.

## Download the datasets

The following cell will download the dataset used by this lab. Click into the following cell and use `Shift + Enter` to execute it

In [1]:
import os

main_path = os.path.abspath(os.path.curdir)
print("Current working directory is ", main_path)
data_path = os.path.join(main_path, 'data')
os.listdir(data_path)

Current working directory is  C:\Users\sasever\Desktop\SelfLearning\AzureML\AML-service-labs-master\starter-artifacts\jupyter\azure-ml-labs\01-model-training


['UsedCars_Affordability.csv', 'UsedCars_Clean.csv']

## Train a simple model

The following cell loads the sampled dataset. Use `Shift + Enter` to execute the cell. Take a moment to look at the data loaded into the Pandas Dataframe - it contains data about used cars such as the price (in dollars), age (in years), KM (kilometers driven) and other attributes like weather it is automatic transimission, the number of doors, and the weight.

In [2]:
# Step 1 - load the data
########################
import numpy as np
import pandas as pd
import os

df = df = pd.read_csv(os.path.join(data_path, 'UsedCars_Clean.csv'), delimiter=',')
print(df)


      Price  Age     KM FuelType   HP  MetColor  Automatic    CC  Doors  \
0     13500   23  46986   Diesel   90         1          0  2000      3   
1     13750   23  72937   Diesel   90         1          0  2000      3   
2     13950   24  41711   Diesel   90         1          0  2000      3   
3     14950   26  48000   Diesel   90         0          0  2000      3   
4     13750   30  38500   Diesel   90         0          0  2000      3   
5     12950   32  61000   Diesel   90         0          0  2000      3   
6     16900   27  94612   Diesel   90         1          0  2000      3   
7     18600   30  75889   Diesel   90         1          0  2000      3   
8     21500   27  19700   Petrol  192         0          0  1800      3   
9     12950   23  71138   Diesel   69         0          0  1900      3   
10    20950   25  31461   Petrol  192         0          0  1800      3   
11    19950   22  43610   Petrol  192         0          0  1800      3   
12    19600   25  32189  

We are going to try and build a model that can answer the question "Can I afford a car that is X months old and has Y kilometers on it, given I have $12,000 to spend?". We will engineer the label for affordable. Execute the following cell.

In [3]:
# Step 2 - add the affordable feature
######################################
df['Affordable'] = np.where(df['Price']<12000, 1, 0)
df_affordability = df[["Age","KM", "Affordable"]]
print(df_affordability)

      Age     KM  Affordable
0      23  46986           0
1      23  72937           0
2      24  41711           0
3      26  48000           0
4      30  38500           0
5      32  61000           0
6      27  94612           0
7      30  75889           0
8      27  19700           0
9      23  71138           0
10     25  31461           0
11     22  43610           0
12     25  32189           0
13     31  23000           0
14     32  34131           0
15     28  18739           0
16     30  34000           0
17     24  21716           0
18     24  25563           0
19     30  64359           0
20     30  67660           0
21     29  43905           0
22     28  56349           0
23     28  32220           0
24     29  25813           0
25     25  28450           0
26     27  34545           0
27     29  41415           0
28     28  44142           0
29     30  11090           0
...   ...    ...         ...
1406   70  44850           1
1407   69  44826           1
1408   80  444

We are going to train a Logistic Regression model in Azure Databricks. This type of model requires us to standardize the scale of our training features Age and KM, so we use the `StandardScaler` from Scikit-Learn to transform these features so that they have values centered with a mean around 0 (mostly between -2.96 and 1.29). Select Step 3 and execute the code. Observe the difference in min and max values between the un-scaled and scaled Dataframes. When we use Sci-Kit Learn, these models are trained on the driver node. Execute the following cell.

In [4]:
# Step 3 - Scale the numeric feature values
###########################################
X = df_affordability[["Age", "KM"]].values
y = df_affordability[["Affordable"]].values

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(pd.DataFrame(X).describe().round(2))
print(pd.DataFrame(X_scaled).describe().round(2))

             0          1
count  1436.00    1436.00
mean     55.95   68533.26
std      18.60   37506.45
min       1.00       1.00
25%      44.00   43000.00
50%      61.00   63389.50
75%      70.00   87020.75
max      80.00  243000.00
             0        1
count  1436.00  1436.00
mean     -0.00     0.00
std       1.00     1.00
min      -2.96    -1.83
25%      -0.64    -0.68
50%       0.27    -0.14
75%       0.76     0.49
max       1.29     4.65


C:\Users\sasever\AppData\Local\Continuum\anaconda3\envs\amlpy36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Train the model by fitting a LogisticRegression against the scaled input features (X_scaled) and the labels (y). Execute the following cell.

In [5]:
# Step 4 - Fit a Logistic Regression
####################################
from sklearn import linear_model
# Create a linear model for Logistic Regression
clf = linear_model.LogisticRegression(C=1)

# we create an instance of Classifier and fit the data.
clf.fit(X_scaled, y)

C:\Users\sasever\AppData\Local\Continuum\anaconda3\envs\amlpy36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Try prediction - if you set the age to 60 months and km to 40,000, does the model predict you can afford the car? Execute the cell and find out.

In [6]:
# Step 5 - Test the trained model's prediction
##############################################
age = 60
km = 40000

scaled_input = scaler.transform([[age, km]])
prediction = clf.predict(scaled_input)

print("Can I afford a car that is {} month(s) old with {} KM's on it?".format(age,km))
print("Yes (1)" if prediction[0] == 1 else "No (0)")

Can I afford a car that is 60 month(s) old with 40000 KM's on it?
Yes (1)


Now, let's get a sense for how accurate the model is. Execute the following cell. What was your model's accuracy?

In [7]:
# Step 6 - Measure the model's performance
##########################################
scaled_inputs = scaler.transform(X)
predictions = clf.predict(scaled_inputs)
print(predictions)

from sklearn.metrics import accuracy_score
score = accuracy_score(y, predictions)
print("Model Accuracy: {}".format(score.round(3)))

[0 0 0 ... 1 1 1]
Model Accuracy: 0.926


C:\Users\sasever\AppData\Local\Continuum\anaconda3\envs\amlpy36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


One thing that can affect the model's performance is how much data of all the labeled training data available is used to train the model. In the next cell, you define a method that uses train_test_split from Scikit-Learn that will enable you to split the data using different percentages. Execute the cell to register this function.

In [8]:
# Step 7 - Define a method to experiment with different training subset sizes
#############################################################################
from sklearn.model_selection import train_test_split
full_X = df_affordability[["Age", "KM"]]
full_Y = df_affordability[["Affordable"]]

def train_eval_model(full_X, full_Y,training_set_percentage):
    train_X, test_X, train_Y, test_Y = train_test_split(full_X, full_Y, train_size=training_set_percentage, random_state=42)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(train_X)
    clf = linear_model.LogisticRegression(C=1)
    clf.fit(X_scaled, train_Y)

    scaled_inputs = scaler.transform(test_X)
    predictions = clf.predict(scaled_inputs)
    score = accuracy_score(test_Y, predictions)

    return (clf, score)

## Use Azure Machine Learning to log performance metrics
In the steps that follow, you will train multiple models using different sizes of training data and observe the impact on performance (accuracy). Each time you create a new model, you are executing a Run in the terminology of Azure Machine Learning service. In this case, you will create one Experiment and execute multiple Runs within it, each with different training percentages (and resultant varying accuracies).

Execute the following cell to quickly verify you have the Azure Machine Learning SDK installed on your cluster. If you get a version number back without error, you are ready to proceed.

In [9]:
# Step 8 - Verify AML SDK Installed
#####################################################################
import azureml.core
print("SDK Version:", azureml.core.VERSION)


SDK Version: 1.0.6


All Azure Machine Learning entities are organized within a Workspace. You can create an AML Workspace in the Azure Portal, but as the code in the following cell shows, you can also create a Workspace directly from code. Set the values for `subscription_id`, `resource_group`, `workspace_name` and `workspace_region` as directed by the comments. Execute Step 9. You will be prompted to log in to your Azure Subscription by the command output.

In [10]:
# Step 9 - Create a workspace
#####################################################################

# Provide the Subscription ID of your existing Azure subscription
subscription_id ='757c4165-0823-49f7-9678-6a79eg8e23cb'

# Provide values for the Resource Group and Workspace that will be created
resource_group = 'MLworkspace2'
workspace_name = 'snml2'
workspace_region = 'westeurope'  # eastus, westcentralus, southeastasia, australiaeast, westeurope

# import the Workspace class
from azureml.core import Workspace

ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

print("Workspace Provisioning complete.")

Workspace Provisioning complete.


You do not need to try to recreate the workspace everytime if you are sure that it exists, insted you can do also this:

In [11]:
ws = Workspace(subscription_id=subscription_id, 
               resource_group=resource_group, 
               workspace_name=workspace_name, 
               auth=None, 
               _location=workspace_region,
               _disable_service_check=False)

In [12]:
ws.get_details()

{'id': '/subscriptions/757c4165-0823-49f7-9678-5a85fe5e17cc/resourceGroups/MLworkspace2/providers/Microsoft.MachineLearningServices/workspaces/snml2',
 'name': 'snml2',
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'workspaceid': 'bc2e7823-cd0c-4764-baf2-9d0a582383d9',
 'description': '',
 'friendlyName': 'snml2',
 'creationTime': '2019-01-29T12:25:04.3559714+00:00',
 'containerRegistry': '/subscriptions/757c4165-0823-49f7-9678-5a85fe5e17cc/resourcegroups/mlworkspace2/providers/microsoft.containerregistry/registries/snml2acrcfvenwfd',
 'keyVault': '/subscriptions/757c4165-0823-49f7-9678-5a85fe5e17cc/resourcegroups/mlworkspace2/providers/microsoft.keyvault/vaults/snml2keyvaultejxdsidr',
 'applicationInsights': '/subscriptions/757c4165-0823-49f7-9678-5a85fe5e17cc/resourcegroups/mlworkspace2/providers/microsoft.insights/components/snml2insightshjyeqgrb',
 'identityPrincipalId': '4f575f4e-a348-4b6d-bffe-f6ebd8408f73',
 'identityTenantId': '72f988bf-86

To begin capturing metrics, you must first create an Experiment and then call `start_logging()` on that Experiment. The return value of this call is a Run. This root run can have other child runs. When you are finished with an experiment run, use `complete()` to close out the root run. Execute the following cell to train four different models using differing amounts of training data and log the results to Azure Machine Learning.

In [13]:
# Step 10 - Create an experiment and log metrics for multiple training runs
###########################################################################
from azureml.core.run import Run
from azureml.core.experiment import Experiment

# start a training run by defining an experiment
myexperiment = Experiment(ws, "UsedCars_Experiment")
root_run = myexperiment.start_logging()

training_set_percentage = 0.25
run = root_run.child_run("Training_Set_Percentage-%0.5F" % training_set_percentage)
model, score = train_eval_model(full_X, full_Y, training_set_percentage)
print("With %0.2f percent of data, model accuracy reached %0.4f." % (training_set_percentage, score))
run.log("Training_Set_Percentage", training_set_percentage)
run.log("Accuracy", score)
run.complete()

training_set_percentage = 0.5
run = root_run.child_run("Training_Set_Percentage-%0.5F" % training_set_percentage)
model, score = train_eval_model(full_X, full_Y, training_set_percentage)
print("With %0.2f percent of data, model accuracy reached %0.4f." % (training_set_percentage, score))
run.log("Training_Set_Percentage", training_set_percentage)
run.log("Accuracy", score)
run.complete()

training_set_percentage = 0.75
run = root_run.child_run("Training_Set_Percentage-%0.5F" % training_set_percentage)
model, score = train_eval_model(full_X, full_Y, training_set_percentage)
print("With %0.2f percent of data, model accuracy reached %0.4f." % (training_set_percentage, score))
run.log("Training_Set_Percentage", training_set_percentage)
run.log("Accuracy", score)
run.complete()

training_set_percentage = 0.9
run = root_run.child_run("Training_Set_Percentage-%0.5F" % training_set_percentage)
model, score = train_eval_model(full_X, full_Y, training_set_percentage)
print("With %0.2f percent of data, model accuracy reached %0.4f." % (training_set_percentage, score))
run.log("Training_Set_Percentage", training_set_percentage)
run.log("Accuracy", score)
run.complete()

# Close out the experiment
root_run.complete()

C:\Users\sasever\AppData\Local\Continuum\anaconda3\envs\amlpy36\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
C:\Users\sasever\AppData\Local\Continuum\anaconda3\envs\amlpy36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


With 0.25 percent of data, model accuracy reached 0.9192.
With 0.50 percent of data, model accuracy reached 0.9109.
With 0.75 percent of data, model accuracy reached 0.9081.
With 0.90 percent of data, model accuracy reached 0.9306.


Now that you have captured history for various runs, you can review the runs. You could use the Azure Portal for this - go to the Azure Portal, find your Azure Machine Learning Workspace, select Experiments and select the UsedCars_Experiment. However, in this case we will use the AML SDK to query for the runs. Execute the following cell to view the runs and their status.

In [14]:
# Step 11 - Review captured runs
################################
# Go to the Azure Portal, find your Azure Machine Learning Workspace, select Experiments and select the UsedCars_Experiment

# You can also query the run history using the SDK.
# The following command lists all of the runs for the experiment
runs = [r for r in root_run.get_children()]
print(runs)

[Run(Experiment: UsedCars_Experiment,
Id: f3d81ed3-89ff-4aba-9d15-a65c2e4acd91,
Type: None,
Status: Completed), Run(Experiment: UsedCars_Experiment,
Id: 48297d82-6769-4511-b830-356bfbe192cb,
Type: None,
Status: Completed), Run(Experiment: UsedCars_Experiment,
Id: 7ad43166-7a19-4bfe-ac54-b701e8f6fa9f,
Type: None,
Status: Completed), Run(Experiment: UsedCars_Experiment,
Id: bdaaa953-2659-4bd8-abb1-6b2b3c14fe08,
Type: None,
Status: Completed)]
